In [ ]:
!pip uninstall -y statsmodels patsy
!pip install patsy==0.5.1
!pip install statsmodels==0.13.2

In [1]:
# Import necessary libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.stats import pearsonr
import statsmodels.api as sm
import ipywidgets as widgets
from IPython.display import display, clear_output

# Define the simulation function
def simulate_betting_market(num_bettors=100, variance_ability=0.1, num_rounds=1000, bet_fraction=0.1):
    # Initialize bettors' abilities and starting wealth
    abilities = np.clip(np.random.normal(0.5, variance_ability, num_bettors), 0, 1)  # Win probabilities between 0 and 1
    wealth = np.ones(num_bettors)  # Start with equal wealth for all bettors

    # To store the weighted average of abilities over time
    expected_values_over_time = []

    # Simulation of betting rounds
    for _ in range(num_rounds):
        # Each bettor places a bet, betting a fraction of their current wealth
        bets = bet_fraction * wealth

        # Calculate the weighted average of abilities based on bet size (wealth-based influence)
        weighted_ability = np.sum(abilities * bets) / np.sum(bets)
        expected_values_over_time.append(weighted_ability)

        # Random outcomes based on abilities
        outcomes = np.random.rand(num_bettors) < abilities

        # Update wealth: win if outcome is True, lose if False
        wealth += bets * (2 * outcomes - 1)

        # Ensure wealth doesn't drop below zero
        wealth = np.maximum(wealth, 0)

    # Create a DataFrame for results
    results = pd.DataFrame({
        "Ability": abilities,
        "End Wealth": wealth
    })

    # Calculate percentile ranks for wealth and ability
    results["Wealth Percentile"] = results["End Wealth"].rank(pct=True) * 100
    results["Ability Percentile"] = results["Ability"].rank(pct=True) * 100

    # Add squared and exponential terms for the ability in the regression
    results["Ability^2"] = results["Ability"] ** 2
    results["Ability_exp"] = np.exp(results["Ability"])

    # Prepare regression model for End Wealth with linear, squared, and exponential terms
    X_wealth = results[["Ability", "Ability^2", "Ability_exp"]]
    X_wealth = sm.add_constant(X_wealth)  # Adds an intercept term to the regression
    y_wealth = results["End Wealth"]
    model_wealth = sm.OLS(y_wealth, X_wealth).fit()  # Ordinary Least Squares regression

    # Prepare regression model for Wealth Percentile with linear, squared, and exponential terms
    y_percentile = results["Wealth Percentile"]
    model_percentile = sm.OLS(y_percentile, X_wealth).fit()

    # Display the regression summaries
    print("Regression Results for End Wealth:\n", model_wealth.summary())
    print("\nRegression Results for Wealth Percentile:\n", model_percentile.summary())

    # Display the correlation between ability and end wealth
    correlation, _ = pearsonr(results["Ability"], results["End Wealth"])
    print("\nCorrelation between ability and end wealth:", correlation)

    # Plotting the results for End Wealth with a log scale on the y-axis
    plt.figure(figsize=(10, 6))
    plt.scatter(results["Ability"], results["End Wealth"], alpha=0.7, label="End Wealth")
    plt.yscale('log')
    plt.xlabel("Ability (Win Probability)")
    plt.ylabel("End Wealth (Log Scale)")
    plt.title("Ability vs. End Wealth After Betting Simulation (Log Scale)")
    plt.show()

    # Plotting the results for Wealth Percentile vs. Ability Percentile
    plt.figure(figsize=(10, 6))
    plt.scatter(results["Ability Percentile"], results["Wealth Percentile"], alpha=0.7, color="orange")
    plt.xlabel("Ability Percentile")
    plt.ylabel("Wealth Percentile")
    plt.title("Ability Percentile vs. Wealth Percentile After Betting Simulation")
    plt.show()

    # Plotting the weighted average of predictive ability over time
    plt.figure(figsize=(10, 6))
    plt.plot(expected_values_over_time, label="Expected Value (Weighted by Bet Size)")
    plt.ylim(0.5, 1)  # Set y-axis range from 0 to 1
    plt.xlabel("Round")
    plt.ylabel("Weighted Average of Predictive Ability")
    plt.title("Weighted Average of Predictive Ability Over Time")
    plt.legend()
    plt.show()

    return results, model_wealth, model_percentile

# Define interactive widgets
num_bettors_slider = widgets.IntSlider(value=100, min=10, max=500, step=10, description='Num Bettors')
variance_ability_slider = widgets.FloatSlider(value=0.1, min=0.01, max=0.5, step=0.01, description='Variance Ability')
num_rounds_slider = widgets.IntSlider(value=1000, min=10, max=2000, step=10, description='Num Rounds')
bet_fraction_slider = widgets.FloatSlider(value=0.1, min=0.01, max=1.0, step=0.01, description='Bet Fraction')
button = widgets.Button(description="Run Simulation")
output = widgets.Output()

# Function to run simulation on button click
def on_button_clicked(b):
    with output:
        clear_output()  # Clear previous output
        simulate_betting_market(
            num_bettors=num_bettors_slider.value,
            variance_ability=variance_ability_slider.value,
            num_rounds=num_rounds_slider.value,
            bet_fraction=bet_fraction_slider.value
        )

# Attach the function to the button
button.on_click(on_button_clicked)

# Display the widgets and output
display(num_bettors_slider, variance_ability_slider, num_rounds_slider, bet_fraction_slider, button, output)

IntSlider(value=100, description='Num Bettors', max=500, min=10, step=10)

FloatSlider(value=0.1, description='Variance Ability', max=0.5, min=0.01, step=0.01)

IntSlider(value=1000, description='Num Rounds', max=2000, min=10, step=10)

FloatSlider(value=0.1, description='Bet Fraction', max=1.0, min=0.01, step=0.01)

Button(description='Run Simulation', style=ButtonStyle())

Output()

Found existing installation: statsmodels 0.13.2
Uninstalling statsmodels-0.13.2:
  Successfully uninstalled statsmodels-0.13.2
Found existing installation: patsy 0.5.2
Uninstalling patsy-0.5.2:
  Successfully uninstalled patsy-0.5.2
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 231.2/231.2 kB 4.8 MB/s eta 0:00:00
  Using cached statsmodels-0.13.2-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (8.1 kB)
Using cached statsmodels-0.13.2-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (10.0 MB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.9/232.9 kB 4.9 MB/s eta 0:00:00
  Attempting uninstall: patsy
    Found existing installation: patsy 0.5.1
    Uninstalling patsy-0.5.1:
      Successfully uninstalled patsy-0.5.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
plotnine 0.14.1 requires statsmodels>=0.14.0, but you have statsmodels 0